In [1]:
import sys
sys.path.append("/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/_scripts")

from case_edits.epcase import EneryPlusCaseEditor
from case_edits.recipes.from_min import create_case

from case_edits.methods.materials import Materials

from case_edits.methods.subsurface import Subsurface, SubsurfaceAttributes, SubsurfaceType
from outputs.variables import OutputVars as ov

from geomeppy.patches import EpBunch

from case_edits.object_getter import Getter

In [2]:
TEST_CASE = "tests/test11_afn"
SURFACE_CONSTRUCTIONS_CASE = "cases/constructions/ASHRAE_2005_HOF_Materials.idf"

In [3]:
outputs = [ov.surf_incident_solar_rad, ov.surf_net_thermal_rad, ov.zone_mean_air_temp, ov.in_surf_temp]
output_names = [o.value for o in outputs]
epcase = create_case(TEST_CASE, outputs=output_names)

cases/tests/test11_afn/out.idf
cases/tests/test11_afn/out.idf
Added 5 walls 
Added 4 walls 


In [4]:
mat = Materials(epcase)
mat.study_reference_idf(SURFACE_CONSTRUCTIONS_CASE)
mat.get_construction_materials('Light Furnishings')
c = mat.get_case_constructions()
# const = mat.case_constructions[-1]

In [5]:
west_surf = epcase.geometry.zones.B_West.walls.B_West_West_W03
int_surf = epcase.geometry.zones.B_West.walls.B_West_North_W02_1
north_surf = epcase.geometry.zones.B_North.walls.B_North_North_W02

window_const = epcase.idf.getobject("CONSTRUCTION", "Project External Window")
door_const = epcase.idf.getobject("CONSTRUCTION", 'Light Furnishings')



In [6]:
# windows
data = SubsurfaceAttributes(type=SubsurfaceType.WINDOW, length=1, height=1, construction=window_const, surface=west_surf) # type: ignore
Subsurface(epcase=epcase,attrs=data).create_surface()

data.surface = north_surf
Subsurface(epcase=epcase,attrs=data).create_surface()

In [7]:
# doors 
data = SubsurfaceAttributes(type=SubsurfaceType.DOOR, length=1, height=2, construction=door_const, surface=int_surf) # type: ignore
Subsurface(epcase=epcase,attrs=data).create_surface()


In [13]:
g = Getter(epcase)
g.get_subsurfaces()


In [19]:
for i in g.subsurfaces:
    try:
        if i.Outside_Boundary_Condition_Object:
            print("hi")
    except:
        pass

hi
hi


In [14]:
item = g.subsurfaces[0]
item.fieldnames

['key',
 'Name',
 'Construction_Name',
 'Building_Surface_Name',
 'Outside_Boundary_Condition_Object',
 'Multiplier',
 'Starting_X_Coordinate',
 'Starting_Z_Coordinate',
 'Length',
 'Height']

In [15]:
item.Name

'Block West Storey 0 Wall 0002_1 Door'

In [11]:
afn = epcase.idf.newidfobject("AirflowNetwork:SimulationControl".upper())
afn


AIRFLOWNETWORK:SIMULATIONCONTROL,
    ,                         !- Name
    NoMultizoneOrDistribution,    !- AirflowNetwork Control
    SurfaceAverageCalculation,    !- Wind Pressure Coefficient Type
    OpeningHeight,            !- Height Selection for Local Wind Pressure Calculation
    LowRise,                  !- Building Type
    500,                      !- Maximum Number of Iterations
    ZeroNodePressures,        !- Initialization Type
    0.0001,                   !- Relative Airflow Convergence Tolerance
    1e-06,                    !- Absolute Airflow Convergence Tolerance
    -0.5,                     !- Convergence Acceleration Limit
    0,                        !- Azimuth Angle of Long Axis of Building
    1,                        !- Ratio of Building Width Along Short Axis to Width Along Long Axis
    No,                       !- Height Dependence of External Node Temperature
    SkylineLU,                !- Solver
    No,                       !- Allow Unsupported Z

In [12]:
afn.Name = "AFN_0"


In [24]:
for v in epcase.geometry.zones.values():
    print(v.name)

Block North Storey 0
Block West Storey 0


In [17]:
default_afn_zone = epcase.idf.newidfobject("AirflowNetwork:MultiZone:Zone".upper())
default_afn_zone


AIRFLOWNETWORK:MULTIZONE:ZONE,
    ,                         !- Zone Name
    NoVent,                   !- Ventilation Control Mode
    ,                         !- Ventilation Control Zone Temperature Setpoint Schedule Name
    0,                        !- Minimum Venting Open Factor
    0,                        !- Indoor and Outdoor Temperature Difference Lower Limit For Maximum Venting Open Factor
    100,                      !- Indoor and Outdoor Temperature Difference Upper Limit for Minimum Venting Open Factor
    0,                        !- Indoor and Outdoor Enthalpy Difference Lower Limit For Maximum Venting Open Factor
    300000,                   !- Indoor and Outdoor Enthalpy Difference Upper Limit for Minimum Venting Open Factor
    ,                         !- Venting Availability Schedule Name
    Standard,                 !- Single Sided Wind Pressure Coefficient Algorithm
    10;                       !- Facade Width

In [18]:
default_afn_zone.Ventilation_Control_Mode = "Constant"
default_afn_zone.Zone_Name

# create one for each object .. 

In [10]:
afn_surf = epcase.idf.newidfobject("AirflowNetwork:MultiZone:Surface".upper())
afn_surf


AIRFLOWNETWORK:MULTIZONE:SURFACE,
    ,                         !- Surface Name
    ,                         !- Leakage Component Name
    ,                         !- External Node Name
    1,                        !- WindowDoor Opening Factor or Crack Factor
    ZoneLevel,                !- Ventilation Control Mode
    ,                         !- Ventilation Control Zone Temperature Setpoint Schedule Name
    0,                        !- Minimum Venting Open Factor
    0,                        !- Indoor and Outdoor Temperature Difference Lower Limit For Maximum Venting Open Factor
    100,                      !- Indoor and Outdoor Temperature Difference Upper Limit for Minimum Venting Open Factor
    0,                        !- Indoor and Outdoor Enthalpy Difference Lower Limit For Maximum Venting Open Factor
    300000,                   !- Indoor and Outdoor Enthalpy Difference Upper Limit for Minimum Venting Open Factor
    ,                         !- Venting Availability 

In [11]:
afn_surf.Surface_Name

''

In [ ]:
afn_surf.Leakage_Component_Name

In [22]:
leakage_component = epcase.idf.newidfobject("AirflowNetwork:MultiZone:Component:SimpleOpening".upper())
leakage_component


AIRFLOWNETWORK:MULTIZONE:COMPONENT:SIMPLEOPENING,
    ,                         !- Name
    ,                         !- Air Mass Flow Coefficient When Opening is Closed
    0.65;                     !- Air Mass Flow Exponent When Opening is Closed

In [ ]:
leakage_component.Discharge_Coefficient = 1
leakage_component.Air_Mass_Flow_Coefficient_When_Opening_is_Closed = 0.001

In [25]:
import re

In [35]:
[SubsurfaceType(i).name for i in range(2)]

['DOOR', 'WINDOW']

In [ ]:
pattern = re.compile("Door".upper())

subsurfaces = []

In [36]:
def get_subsurface_by_type(type):
    for k,v in epcase.idf.idfobjects.items():
        if bool(pattern.match(k)):
            if len(v) > 0:
                for item in v:
                    subsurfaces.append(item)


In [32]:
subsurfaces

[
 DOOR:INTERZONE,
     Block West Storey 0 Wall 0002_1 Door,    !- Name
     Light Furnishings,        !- Construction Name
     Block West Storey 0 Wall 0002_1,    !- Building Surface Name
     Block North Storey 0 Wall 0004_1 Door,    !- Outside Boundary Condition Object
     1.0,                      !- Multiplier
     2,                        !- Starting X Coordinate
     0.02,                     !- Starting Z Coordinate
     1,                        !- Length
     2;                        !- Height,
 
 DOOR:INTERZONE,
     Block North Storey 0 Wall 0004_1 Door,    !- Name
     Light Furnishings,        !- Construction Name
     Block North Storey 0 Wall 0004_1,    !- Building Surface Name
     Block West Storey 0 Wall 0002_1 Door,    !- Outside Boundary Condition Object
     1.0,                      !- Multiplier
     2,                        !- Starting X Coordinate
     0.02,                     !- Starting Z Coordinate
     1,                        !- Length
     2;    

In [8]:
# epcase.run_idf()


/Applications/EnergyPlus-22-2-0/energyplus --weather /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/weather/CA_PALO-ALTO-AP_724937S_20.epw --output-directory /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/cases/tests/test11_afn/results --idd /Applications/EnergyPlus-22-2-0/Energy+.idd /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/cases/base/01example/Minimal_45a2c6.idf

EnergyPlus Starting
EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.06.17 16:42
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Reporting Surfaces
Beginning Primary Simulation
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 12/21 for DENVER_STAPLETON ANN HTG 99.6% CONDNS DB
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 07/21 for DENVER_S

EnergyPlus Completed Successfully.
